In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Change to the desired directory in Google Drive
os.chdir('/content/drive/My Drive/Colab Notebooks/Mental Health Depression Bertopic')

# Verify by listing files in the current directory
print(os.listdir())

Mounted at /content/drive
['data', 'my_model_dir', 'Topic_modeling_BERTopic.ipynb', 'Sentiment Dsitribution and Topic Analysis.ipynb', 'previous reddir small data', 'Depression Data Cleaning and Preprocess', 'DepressionRobertaSentimentAnalysis']


In [ ]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Mental Health Depression Bertopic/data/preprocessed_depression_data.csv')

In [ ]:
data.isna().sum()

,0
clean_text,1
is_depression,0
text,0


In [ ]:
data.dropna(inplace=True)

In [ ]:
data.head()

,clean_text,is_depression,text
0,understand people reply immediately op invitat...,1,we understand that most people who reply immed...
1,welcome r depression check post place take mom...,1,welcome to r depression s check in post a plac...
2,anyone else instead sleeping depressed stay ni...,1,anyone else instead of sleeping more when depr...
3,kind stuffed around lot life delaying inevitab...,1,i ve kind of stuffed around a lot in my life d...
4,sleep greatest comforting escape whenever wake...,1,sleep is my greatest and most comforting escap...


In [ ]:
data['is_depression'].value_counts()

,count
is_depression,
0,3900
1,3830


## Regenerate the previous work

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import csv
import urllib.request
import torch

In [ ]:
sns.set(font_scale=1)
warnings.filterwarnings('ignore')
torch.cuda.device_count()

1

The following part is done differently with a different but similar reddit data.

In [ ]:
"""
Choose the ids from user_ids file and export data from SMHD paper. then select the rows with word count <3 to >200 only.
"""
# read the data
#data = pd.read_csv("depression.csv") # this data file is the selected 1316 users from depression group and 1316 users from control group which has word count greater than 3 and less then 200.
print("shape of df:",data.shape)

shape of df: (7730, 3)


In [ ]:
# Roberta sentiment model and tokenizer
sentiment_model_path = f"cardiffnlp/twitter-roberta-base-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_path)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Since the url doensn't work the following code will not be run. sentiment_labels will be defined as positive, negative and neutral.

In [ ]:
# Download label mapping
sentiment_labels = []
sentiment_mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/mapping.txt"
with urllib.request.urlopen(sentiment_mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
sentiment_labels = [row[1] for row in csvreader if len(row) > 1]

In [ ]:
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_path)

#sentiment_model.save_pretrained(sentiment_model_path)
#sentiment_tokenizer.save_pretrained(sentiment_model_path)

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
if torch.cuda.is_available():
  print("GPU is available!")
  device = torch.device('cuda')
else:
  print("GPU is not available.")
  device = torch.device('cpu')

GPU is available!


In [ ]:
#device = torch.device('cuda')
sentiment_model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
#Customer dataloader
class TextDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data['id'].iloc[idx], self.data['text'].iloc[idx], self.data['clean_text'].iloc[idx], self.data['label'].iloc[idx], self.data['created_utc'].iloc[idx]

In [ ]:
# BATCH SIZE CALCULATION
def find_optimal_batch_size(data_size, max_batch_size=512):
    """Find an optimal batch size such that data_size/batch_size is a whole number."""

    for batch_size in range(max_batch_size, 0, -1):
        if data_size % batch_size == 0:
            return batch_size
    return 1  # Return 1 if no optimal batch size is found up to max_batch_size
dataset = TextDataset(data)
data_size = len(data)
optimal_batch_size = find_optimal_batch_size(data_size)

optimal_batch_size
train_dataloader = DataLoader(dataset, batch_size=optimal_batch_size, shuffle=False,drop_last=False) # Shuffle should be set to False

In our data we do not have id column. So we will set index column as id column to run the next part of the code.

In [ ]:
data['id'] = data.index
data['label'] = 0
data['created_utc'] = 0

In [ ]:
# 1. Compute the total number of rows in the original DataFrame
original_data_count = len(data)

# 2. Iterate through the DataLoader and count the total number of items processed
dataloader_count = sum(len(batch[0]) for batch in train_dataloader)  # Assuming batch[0] contains the 'id' or 'text'

# 3. Compare the two counts
is_all_data_processed = original_data_count == dataloader_count

print(is_all_data_processed)

True


The the url cannot be reached we use the following sentiment labels as a best guess.

In [ ]:
sentiment_labels =["negative", "neutral", "positive"]

In [ ]:
user = ["depression" if row else "control" for row in data['is_depression'] ]

In [ ]:
# Ensure the model is in evaluation mode
sentiment_model.eval()

# List to store the results as dictionaries
results_list = []
# Process each batch from the dataloader
for ids, doc,clean_texts, labels,utc in train_dataloader:

    # Convert tensors to appropriate data structures
    doc_list =list(doc)
    utc_list = list(utc)
    labels_list = list(labels)
    texts_list = list(clean_texts)


    # Tokenize and move data to GPU
    encoded_input = sentiment_tokenizer(texts_list, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

    # Calculate sentiment
    with torch.no_grad():
        output = sentiment_model(**encoded_input)

    # Compute the softmax to get the confidence scores
    scores = torch.nn.functional.softmax(output.logits, dim=1)

    # Extract confidence scores for each sentiment label and determine the max label
    for idm, doc,clean_text_,label_,created_utc_, score in zip(ids, doc_list,texts_list,labels_list,utc_list, scores):

        # Determine the sentiment label with the highest confidence
        max_idx = torch.argmax(score).item()
        max_label = sentiment_labels[max_idx]

        # Create a dictionary for this row and append to the results list
        row_dict = {
            'id': idm.item(),
            'label':label_,
            'document':doc,
            'created_utc':created_utc_.item(),
            'clean_text': clean_text_,
            'negative': score[0].item(),
            'neutral': score[1].item(),
            'positive': score[2].item(),
            'sentiment': max_label,
        }
        results_list.append(row_dict)


In [ ]:
# Convert the results list to a DataFrame
results_df = pd.DataFrame(results_list)
# add users column
results_df['user'] = user
# Display the new DataFrame with predictions
results_df.head()
results_df.shape,data.shape
data.label.value_counts(), data.groupby('label')['id'].nunique()
results_df.label.value_counts(), results_df.groupby('label')['id'].nunique()
results_df.to_csv("depression_sentiments_roberta.csv",index=False) # this is the next data file to be used for topic modeling containing original data +sentiment labels

In [ ]:
results_df.head()

,id,label,document,created_utc,clean_text,negative,neutral,positive,sentiment,user
0,0,tensor(0),we understand that most people who reply immed...,0,understand people reply immediately op invitat...,0.123658,0.683168,0.193175,neutral,depression
1,1,tensor(0),welcome to r depression s check in post a plac...,0,welcome r depression check post place take mom...,0.265393,0.642364,0.092243,neutral,depression
2,2,tensor(0),anyone else instead of sleeping more when depr...,0,anyone else instead sleeping depressed stay ni...,0.225754,0.600676,0.173570,neutral,depression
3,3,tensor(0),i ve kind of stuffed around a lot in my life d...,0,kind stuffed around lot life delaying inevitab...,0.868262,0.117783,0.013955,negative,depression
4,4,tensor(0),sleep is my greatest and most comforting escap...,0,sleep greatest comforting escape whenever wake...,0.355922,0.439947,0.204132,neutral,depression


In [ ]:
results_df.columns

Index(['id', 'label', 'document', 'created_utc', 'clean_text', 'negative',
       'neutral', 'positive', 'sentiment', 'user'],
      dtype='object')